In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
print(keras.__version__)

In [ ]:
tf.enable_eager_execution()

In [ ]:
sentence = " if you want you"

idx2char = list(set(sentence))
char2idx = {w: i for i, w in enumerate(idx2char)}
print(idx2char)
print(char2idx)

In [ ]:
data_dim = len(idx2char)
hidden_size = len(idx2char)
num_classes = len(idx2char)
sequence_length = 10  # Any arbitrary number
batch_size = 1
learning_rate = 0.1
training_epochs = 30
print(num_classes)

In [ ]:
sentence_idx = [char2idx[c] for c in sentence]
dataX = [sentence_idx[:-1]]
dataY = [sentence_idx[1:]]

In [ ]:
dataX = np.array(to_categorical(dataX, num_classes))
dataY = np.array(to_categorical(dataY, num_classes))
print(dataX.shape, dataY.shape)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dataX, dataY)).shuffle(
                buffer_size=10).prefetch(buffer_size=batch_size).batch(batch_size)

In [ ]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=num_classes, activation=keras.layers.ReLU(),
                                input_shape=(15,10)))
    model.add(keras.layers.Dense(units=num_classes))
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
def loss_fn(model, dataX, dataY):
    logits = model(dataX, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=dataY))    
    return loss 

In [ ]:
def grad(model, dataX, dataY):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, dataX, dataY)
    return tape.gradient(loss, model.variables)

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [ ]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.    
    train_step = 0    
    
    for X, Y in train_dataset:
        grads = grad(model, X, Y)                
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, X, Y)        
        avg_loss = avg_loss + loss        
        train_step += 1
    avg_loss = avg_loss / train_step
    
    #results = model(dataX, training=False).numpy()
    #results = np.argmax(results, axis=-1)
    #result_str = [idx2char[c] for c in np.squeeze(results)]
    print('Prediction:', ''.join(result_str))
    if (epoch+1) % 10 == 0:
        print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss))          

print('Learning Finished!')

In [ ]:
results = model(dataX, training=False)
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([idx2char[t] for t in index]), end='')
    else:
        print(idx2char[index[-1]], end='')